In [0]:
%sql
CREATE DATABASE IF NOT EXISTS desafio_sql_prova;
USE desafio_sql_prova;
SELECT current_database(); 

current_database()
desafio_sql_prova


#Desafio 1.1: Campeonato

## Calcule o número total de pontos que cada equipe marcou.
- Vitória (Gols marcados > Gols sofridos): 3 pontos 
- Empate (Gols marcados = Gols sofridos): 1 ponto 
- Derrota (Gols marcados < Gols sofridos): 0 pontos

 A tabela deve ser ordenada por num_pontos em ordem decrescente, e em caso de empate, por time_id

In [0]:
%sql
CREATE TABLE IF NOT EXISTS times (
  time_id INTEGER NOT NULL,
  time_nome VARCHAR(255) NOT NULL
);

INSERT INTO times VALUES
  (10, 'Financeiro'),
  (20, 'Marketing'),
  (30, 'Logística'),
  (40, 'TI'),
  (50, 'Dados');

num_affected_rows,num_inserted_rows
5,5


In [0]:
CREATE TABLE IF NOT EXISTS jogos (
  jogo_id INTEGER NOT NULL,
  mandante_time INTEGER NOT NULL,
  visitante_time INTEGER NOT NULL,
  mandante_gols INTEGER NOT NULL,
  visitante_gols INTEGER NOT NULL
);

INSERT INTO jogos VALUES
  (1, 30, 20, 1, 0),
  (2, 10, 20, 1, 2),
  (3, 20, 50, 2, 2),
  (4, 10, 30, 1, 0),
  (5, 30, 50, 0, 1);

num_affected_rows,num_inserted_rows
5,5


##Resultado

In [0]:
WITH PontosPorJogo AS (
    SELECT
        mandante_time AS time_id,
        CASE
            WHEN mandante_gols > visitante_gols THEN 3  
            WHEN mandante_gols = visitante_gols THEN 1 
            ELSE 0                                      
        END AS pontos
    FROM jogos
    
    UNION ALL
    
    SELECT
        visitante_time AS time_id,
        CASE
            WHEN visitante_gols > mandante_gols THEN 3  
            WHEN visitante_gols = mandante_gols THEN 1  
            ELSE 0                                   
        END AS pontos
    FROM jogos
)

SELECT
    t.time_nome, SUM(p.pontos) AS num_pontos
FROM PontosPorJogo AS p
JOIN times AS t
    ON p.time_id = t.time_id
GROUP BY
    t.time_id, t.time_nome
ORDER BY
    num_pontos DESC,  
    t.time_id ASC;    

time_nome,num_pontos
Marketing,4
Dados,4
Financeiro,3
Logística,3


#Desafio 1.2: Comissões

##Retorne a lista de vendedores que receberam até R$ 1024 em até três transferências.

In [0]:
CREATE TABLE IF NOT EXISTS comissoes (
  comprador VARCHAR(255) NOT NULL,
  vendedor VARCHAR(255) NOT NULL,
  dataPgto DATE NOT NULL,
  valor FLOAT NOT NULL
);

INSERT INTO comissoes (comprador, vendedor, dataPgto, valor) VALUES
  ('Leonardo', 'Bruno', '2000-01-01', 200.00),
  ('Leonardo', 'Matheus', '2003-09-27', 1024.00),
  ('Leonardo', 'Lucas', '2006-06-26', 512.00),
  ('Marcos', 'Lucas', '2020-12-17', 100.00),
  ('Marcos', 'Lucas', '2002-03-22', 10.00),
  ('Cinthia', 'Lucas', '2021-03-20', 500.00),
  ('Mateus', 'Bruno', '2007-06-02', 400.00),
  ('Mateus', 'Bruno', '2006-06-26', 400.00),
  ('Mateus', 'Bruno', '2015-06-26', 200.00);

num_affected_rows,num_inserted_rows
9,9


In [0]:
USE desafio_sql_prova;
WITH VendasOrdenadas AS (
    SELECT
        vendedor,
        valor,
        SUM(valor) OVER (
            PARTITION BY vendedor     
            ORDER BY valor DESC     
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW 
        ) AS soma_acumulada,
        
        ROW_NUMBER() OVER (
            PARTITION BY vendedor
            ORDER BY valor DESC
        ) AS num_transf
    FROM comissoes
)

SELECT DISTINCT
    vendedor
FROM VendasOrdenadas
WHERE 
    num_transf <= 3               
    AND soma_acumulada >= 1024   
ORDER BY
    vendedor ASC;

vendedor
Lucas
Matheus


#Desafio 1.3: Organização Empresarial

##Para cada funcionário, encontre o chefe indireto de classificação mais baixa na hierarquia que ganha pelo menos o dobro do salário do funcionário.

- Regra de Hierarquia: O funcionário A é mais baixo na hierarquia do que B se A tem mais chefes indiretos do que B.

- Requisito de Salário: O chefe indireto (o candidato) deve ter salário >= 2 * salário do funcionário.

Objetivo: Encontrar o chefe que satisfaça o requisito salarial e tenha a menor contagem de chefes indiretos.

In [0]:
CREATE TABLE IF NOT EXISTS colaboradores (
  id INTEGER NOT NULL,
  nome VARCHAR(255) NOT NULL,
  salario INTEGER NOT NULL,
  lider_id INTEGER 
); --não coloquei o UNIQUE

INSERT INTO colaboradores (id, nome, salario, lider_id) VALUES
  (40, 'Helen', 1500, 50),
  (50, 'Bruno', 3000, 10),
  (10, 'Leonardo', 4500, 20),
  (20, 'Marcos', 10000, NULL),
  (70, 'Mateus', 1500, 10),
  (60, 'Cinthia', 2000, 70),
  (30, 'Wilian', 1501, 50);

num_affected_rows,num_inserted_rows
7,7


In [0]:
WITH RECURSIVE Hierarquia AS (
    SELECT c.id AS colaborador_id, c.lider_id AS chefe_id, 1 AS nivel
    FROM colaboradores AS c
    WHERE c.lider_id IS NOT NULL
    UNION ALL
    SELECT h.colaborador_id, c.lider_id AS chefe_id, h.nivel + 1 AS nivel
    FROM Hierarquia h
    JOIN colaboradores c ON h.chefe_id = c.id
    WHERE c.lider_id IS NOT NULL
),
ChefesValidos AS (
    SELECT h.colaborador_id, h.chefe_id, h.nivel,
        CASE WHEN ch.salario >= 2 * c.salario THEN 1 ELSE 0
        END AS atende
    FROM Hierarquia h
    JOIN colaboradores c ON h.colaborador_id = c.id
    JOIN colaboradores ch ON h.chefe_id = ch.id
),
Rankeado AS (
    SELECT colaborador_id, chefe_id, nivel, atende,
        ROW_NUMBER() OVER (
            PARTITION BY colaborador_id
            ORDER BY CASE WHEN atende = 1 THEN 0 ELSE 1 END, nivel ASC
        ) AS rn
    FROM ChefesValidos
)

SELECT
    c.id AS colaborador_id,
    CASE WHEN r.atende = 1 THEN r.chefe_id ELSE NULL END AS chefe_selecionado
FROM colaboradores c
LEFT JOIN Rankeado r ON c.id = r.colaborador_id AND r.rn = 1
ORDER BY c.id;


colaborador_id,chefe_selecionado
10,20
20,null
30,10
40,50
50,20
60,10
70,10
